In [1]:
%reload_ext autoreload
%autoreload 2

import time
from tqdm import tqdm
import numpy as np
import scipy as sp
import torch
import torch.nn.functional as F
from torch import nn
from torchvision.transforms import v2 as transforms

from matplotlib import pyplot as plt

from networkAlignmentAnalysis.models.models import MLP, CNN2P2, AlexNet
from networkAlignmentAnalysis import utils
from networkAlignmentAnalysis import datasets
from networkAlignmentAnalysis import files

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device: ', DEVICE)

using device:  cuda


In [ ]:
# TODO
# 1.1. include additional AlignmentModel methods stored in extra class in base model
# -- the measure_eigenfeature method is baaaaad -- because it doesn't account for convolutional layers

# I should also break apart the measure_eigenfeatures method for better parameter handling
# and also to allow use of components of the method in other contexts

# 2. Make CIFAR class
# 4. Rewrite existing analysis pipelines
# 5. SLURM!!!!


# Analyses:
# - targeted dropout throughout training curves (blue and black in alignmentMNIST.ipynb)
# - compare initial to final alignment...
# - compare initial alignment to delta weight norm...
# - observe alignment of weight norm
# - compare alignment to outgoing weight norm!

In [124]:

for outer in tqdm([10, 20, 30, 40, 50], desc=" outer", position=0):
    for inner in tqdm(range(outer), desc=" inner loop", position=1, leave=False):
        time.sleep(0.05)
print("done!")


 outer: 100%|██████████| 5/5 [00:09<00:00,  1.87s/it]

done!


In [126]:
from tqdm.notebook import tqdm
import time

# Example of outer and inner loops
outer_iterations = 5
inner_iterations = 10

# Outer loop (using tqdm)
for i in tqdm(range(outer_iterations), desc='Outer Loop'):
    # Inner loop (using tqdm as well)
    for j in tqdm(range(inner_iterations), desc='Inner Loop', leave=False):
        # Simulating some work
        time.sleep(0.1)

# Close the tqdm bars to display final results
tqdm.write("Finished!")


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html